# Tensor是什么
torch中的tensor类似numpy的ndarrays, 是最基本的数据结构，用于存储和操作多维数组。

In [54]:
import torch
import numpy as np


___

### 创建tensor

In [55]:
# 列表到张量
tensor_from_list = torch.tensor([1,2,30])
# ndarray 搭配张量
np_array = np.array([4,5,6])
tensor_from_numpy = torch.tensor(np_array)
print(tensor_from_numpy)

tensor([4, 5, 6], dtype=torch.int32)


In [56]:
# 全零
zero_tensor = torch.zeros((3, 4))
print(zero_tensor)
# 全1
ones_tensor = torch.ones((2,2))
print(ones_tensor)
# 指定范围张量
range_tensor = torch.arange(0, 10, 2)
print(range_tensor)
# 均匀分布的tensor
mean_tensor = torch.rand((3,3))
print(mean_tensor)
# 正态分布tensor
normal_tensor = torch.randn((3,3))
print(normal_tensor)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([0, 2, 4, 6, 8])
tensor([[0.4749, 0.3585, 0.0671],
        [0.4649, 0.5255, 0.5668],
        [0.1080, 0.7678, 0.1878]])
tensor([[ 0.3145,  0.0897,  0.7994],
        [ 1.1631, -0.3329,  1.0968],
        [ 0.3863, -0.2104,  1.1247]])


In [57]:
# 未初始化的tensor
empty_tensor = torch.empty((2,2))

# 常见与某个tensor形状相同的tensor
same_shape_tensor = torch.ones_like(normal_tensor)
print(empty_tensor)
print(same_shape_tensor)

tensor([[2.3694e-38, 0.0000e+00],
        [3.0000e+00, 3.0000e+00]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])



___

## 张量的基本操作

In [58]:
# 张量的尺寸
# tensor_from_list = torch.tensor([1,2,30])
shape_tensor = tensor_from_list.shape
print(shape_tensor)

# 索引
element = tensor_from_list[0]
print(element)

# 切片
sliced_tensor = tensor_from_list[:2]
print(sliced_tensor)

torch.Size([3])
tensor(1)
tensor([1, 2])


In [59]:
# 改变张量的形状
reshaped_tensor = tensor_from_list.view(1,3)
# 改成1，3相当于从一维张量转为了二维张量
print(reshaped_tensor)
# 张量转置
transposed_tensor = tensor_from_list.t()
print(transposed_tensor)

tensor([[ 1,  2, 30]])
tensor([ 1,  2, 30])


**数学计算**

In [60]:
tensor_o = torch.ones(1,3)
tensor_rand = torch.randn(1,3)
print(tensor_o, tensor_rand)

#加法操作
tensor_sum = tensor_o + tensor_rand
print("sum:", tensor_sum)

# 乘法操作: 注意matmul是典型的矩阵乘法
product_tensor = torch.matmul(tensor_sum, tensor_o.t())
print(product_tensor)
product_tensor_2 = torch.matmul(tensor_sum.t(), tensor_o)
print(product_tensor_2)

# 广播计算
tensor_o_broad = tensor_o * 3
print(tensor_o_broad)
tensor_o_broad_2 = tensor_o_broad + 2
print(tensor_o_broad_2)

tensor([[1., 1., 1.]]) tensor([[-0.4956, -1.4897,  1.7172]])
sum: tensor([[ 0.5044, -0.4897,  2.7172]])
tensor([[2.7319]])
tensor([[ 0.5044,  0.5044,  0.5044],
        [-0.4897, -0.4897, -0.4897],
        [ 2.7172,  2.7172,  2.7172]])
tensor([[3., 3., 3.]])
tensor([[5., 5., 5.]])


## 自动求导


# 自动求导
pytorch自动求导的核心是torch.autograd模块，它为对张量的所有操作提供了自动求导服务，其核心部分包含:
+ tensor
+ 计算图
+ 梯度计算
+ 梯度累计
+ 离散跟踪

如果张量的requires_grad属性被设置为True，那么对它的每一步操作都将被跟踪。当结束计算后，可以调用**backward()**方法，来自动计算所有参数的梯度，注意这个梯度是**累加的**

In [80]:
x = torch.randn(3, 3, requires_grad = True)
print(x.grad_fn)
# 由于x是用户创建的叶子节点，因此没有跟踪计算图

None


In [81]:
x = torch.ones(2,2, requires_grad= True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [82]:
# 对张量进行一次计算
y = x ** 2
print(y)
print(y.grad_fn)
# more
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[1., 1.],
        [1., 1.]], grad_fn=<PowBackward0>)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<MulBackward0>) tensor(3., grad_fn=<MeanBackward0>)


由于y是对x进行操作的结果，因此这个操作会被跟踪，y的grad_fn属性为grad_fn=<PowBackward0>, 字面上看是幂计算反向传播。

In [83]:
# requires_grad属性默认是false
a = torch.randn(3,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
# .requires_grad_()方法可以原地修改张量的requires_grad属性
a.requires_grad_(True)
b = (a * a).sum()
print(b.grad_fn)

False


___

## 梯度

In [84]:
# 对out反向传播
out.backward()

In [85]:
# 看看d(out)/dx
print(x)
print(x.grad)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]])


In [86]:
# 雅可比向量积的例子:
j = torch.randn(3, requires_grad=True)
print(j)

k = j * 2
m = 0
while k.data.norm() < 1000:
    k = k * 2
    m = m + 1
print(k)
print(m)
# 此时k已经不再是标量，autograd不能直接计算梯度，因此只可以求雅可比向量积。
v = torch.tensor([0.1, 1.0, 0.0001],dtype=torch.float)
k.backward(v)
print(j.grad)

tensor([-0.3160,  0.9796, -2.0975], requires_grad=True)
tensor([ -161.7708,   501.5692, -1073.9320], grad_fn=<MulBackward0>)
8
tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


注意，auto_grad只能直接求**标量**结果的反向传播结果。如果结果是向量的话，对所有输入求导就会得到一个**雅可比矩阵**，而不是该输入的梯度值。因此，如果最终结果是向量的话，backward()需要引入一个与输出同维度的权重向量。

___

***梯度的累加机制***

In [87]:
# 如果不手动对参数的梯度清零的话，其梯度会一直累加
out2 = x.sum()
out2.backward()
print(x.grad)
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[4., 4.],
        [4., 4.]])
tensor([[1., 1.],
        [1., 1.]])


___

## 离散追踪
推理或者验证模型的时候，我们不希望数值计算过程被追踪，因为那样会消耗额外的内存与算力。此时使用torch.no_grad()来临时将tensor的requires_grad属性设置为False。

In [88]:
print(j.requires_grad)
print((j ** 2).requires_grad)

with torch.no_grad():
    print((j ** 2).requires_grad)

True
True
False


___
tip:如果我们希望改变一个tensor的值，却不想该百年它的autograd记录，那么就直接对tensor.data属性进行操作：

In [89]:
x = torch.ones(1, requires_grad=True)

print(x.data)  # 其实还是一个tensor
print(x.data.requires_grad)

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x)
print(x.grad)  # 发现导数为2， 与100无关

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])


___

## grad_fn属性详解

当我们对张量进行运算的时候，得到的张量的grad_fn会自动记录该操作，**并指向一个Function对象**，该对象复制前向传播和反向传播。tensor与Function共同组成一个无环有向的计算图。



# 多gpu并行计算

### 什么是CUDA?
CUDA是英伟达GPU的并行运算框架。对GPU的编程是使用CUDA语言完成的。然而pytorch的cuda意思是我们即将使用GPU来处理数据和模型。

当我们希望把模型或者数据从cpu迁移到gpu时，就需要使用.cuda()方法（默认0号gpu）。
tips：
+ pytorch目前不支持amd的opengl接口
+ 避免频繁地将数据在cpu和gpu之间切换
+ 进行简单操作时，使用cpu

——————

如何设置默认显卡？两种方法


In [ ]:
 #设置在文件最开始部分
import os
os.environ["CUDA_VISIBLE_DEVICE"] = "2" # 设置默认的显卡
# 或者
CUDA_VISBLE_DEVICE=0,1 python train.py # 使用0，1两块GPU

## 常见的并行训练方法：

**1.mambaout,模型拆分**  
![模型拆分图解](https://datawhalechina.github.io/thorough-pytorch/_images/model_parllel.png)  
该方法难在gpu之间的通信

**2.同一层任务分不到不同数据中**
这种方法将同一层模型做拆分，不太理解  

**3.不同数据拆分到不同设备(数据并行方式, data parallelism)**  
![](https://datawhalechina.github.io/thorough-pytorch/_images/data_parllel.png)  
该方法的逻辑是将相同的模型复制到各个显卡中，然后将数据切分，让各个显卡训练数据的一部分，最后进行汇总反向传播。这样可以解决通信问题。

___


### 多卡训练 
pytorch提供了data parallel(DP)方式和DistributedDataParallel(DDP)两种多卡训练方式

***使用CUDA加速训练***  
**单卡训练**  
非常简单，只要将模型和数据.cuda()就行了  
首先要手动指定对程序可见的gpu

**多卡DP**

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

In [ ]:
model = Net()
model.cuda() # 模型显示转移到CUDA上

if torch.cuda.device_count() > 1: # 含有多张GPU的卡
	model = nn.DataParallel(model) # 单机多卡DP训练

DP的抽象过程，数据被分为多个子集后，分别在各个gpu计算梯度，最后把梯度汇总到一个主gpu上进行参数的更新，这样会使得主gpu的工作负担明显大于其他gpu，成为性能瓶颈。此外，DP只适合单机模式。**注意DP是单进程多线程**

问题：既然python解释器有GIL锁，为什么限制不了DP的加速？
因为pytorch数据运行于gpu上，由CUDA和其他GPU加速库管理，不受python解释器管理。

**多机多卡DDP**  
DDP是一种多卡多进程方式，每个进程独立运行在一个gpu上，每个gpu独立更新参数，使用高效的通信机制进行梯度信息同步。

## GIL锁对于DP与DDP的影响？
由于模型训练大部分内容是在gpu上运行的，因此对于DP，GIL锁对工作的大部分内容几乎没有影响。
但是对于**运行于cpu的数据预处理加载和IO操作，还有数据后处理和日志记录等**，可能成为性能瓶颈。
但是对于多进程的DDP，由于各个进程都有独立的python解释器，实现了真正的并行运行，因此不受GIL影响。

这对于严重依赖 Python runtime 的 models 而言，比如说包含 RNN 层或大量小组件的 models 而言，这尤为重要。什么是python runtime？？？

# 深度学习训练的整体流程
+ 数据预处理： 数据格式统一、异常数据清除、数据变换
+ 划分训练集、验证集、测试集（可以使用sklearn自带的test_train_split函数）
+ 模型选择、损失函数、优化方法
+ 超参数设置
+ 使用模型拟合训练数据集，在验证集、测试机上评估模型表现



## 包的导入

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

NameError: name '_C' is not defined